In [1]:
pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("Gemini API key setup complete")

except Exception as e:
    print("Authentication error : {e}")

Gemini API key setup complete


In [3]:
import json
import requests
import subprocess
import time
import uuid

from datetime import datetime, timedelta
from typing import Dict, List

from google.adk.agents import LlmAgent
from google.adk.runners import InMemoryRunner
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

#hide additional warning in the notebook
import warnings

warnings.filterwarnings("ignore")

print("ADK imported successfully")

ADK imported successfully


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [5]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        if(
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
            
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print("Generated Python code >>", response_code["result"].replace("tool_code", ""),
                         )
                    
                else:
                    print("Generated Python response >> ", response_code["result"])

print("Helper function defined")

Helper function defined


In [6]:

import pandas as pd
import numpy as np




class BusinessDataManager:
    """Manages sample data generation and storage"""
    
    def __init__(self):
        self.sales_data = None
        self.price_data = None
        self.load_or_generate_data()
    
    def load_or_generate_data(self):
        """Load existing data or generate new sample data"""
        try:
            if os.path.exists('sample_sales_data.csv') and os.path.exists('sample_price_data.csv'):
                self.sales_data = pd.read_csv('sample_sales_data.csv')
                self.price_data = pd.read_csv('sample_price_data.csv')
                print("✅ Loaded existing sample data")
            else:
                self.generate_sample_data()
                print("✅ Generated new sample data")
                
            # Convert date columns
            self.sales_data['date'] = pd.to_datetime(self.sales_data['date'])
            self.price_data['date'] = pd.to_datetime(self.price_data['date'])
            
        except Exception as e:
            print(f"❌ Error loading data: {e}")
            self.generate_sample_data()
    
    def generate_sample_data(self):
        """Generate realistic sample sales and price data"""
        # Product catalog
        products = [
            {'product_id': 'PROD-001', 'product_name': 'Wireless Bluetooth Headphones', 'category': 'Electronics', 'base_price': 79.99},
            {'product_id': 'PROD-002', 'product_name': 'Smart Fitness Watch', 'category': 'Electronics', 'base_price': 199.99},
            {'product_id': 'PROD-003', 'product_name': 'Organic Cotton T-Shirt', 'category': 'Clothing', 'base_price': 24.99},
            {'product_id': 'PROD-004', 'product_name': 'Stainless Steel Water Bottle', 'category': 'Home', 'base_price': 29.99},
            {'product_id': 'PROD-005', 'product_name': 'Gaming Mechanical Keyboard', 'category': 'Electronics', 'base_price': 129.99},
        ]
        
        # Generate 180 days of data
        end_date = datetime.now()
        start_date = end_date - timedelta(days=180)
        date_range = pd.date_range(start=start_date, end=end_date, freq='D')
        
        sales_records = []
        price_records = []
        
        for date in date_range:
            for product in products:
                # Generate sales data with trends and seasonality
                base_quantity = {
                    'PROD-001': 12, 'PROD-002': 6, 'PROD-003': 20, 'PROD-004': 15, 'PROD-005': 8
                }[product['product_id']]
                
                # Seasonality and trends
                day_of_week = date.weekday()
                weekend_boost = 1.4 if day_of_week >= 5 else 1.0
                month_progress = (date - start_date).days / 180
                
                # Product-specific trends
                trend_factors = {
                    'PROD-001': 1.1,  # Growing
                    'PROD-002': 1.2,  # Fast growth
                    'PROD-003': 0.95, # Slight decline
                    'PROD-004': 1.0,  # Stable
                    'PROD-005': 1.05  # Slow growth
                }
                
                trend = 1 + (trend_factors[product['product_id']] - 1) * month_progress
                random_factor = np.random.normal(1.0, 0.15)
                
                quantity = max(1, int(base_quantity * weekend_boost * trend * random_factor))
                
                # Price data with fluctuations
                base_price = product['base_price']
                price_variation = np.random.normal(1.0, 0.08)  # 8% variation
                current_price = max(base_price * 0.7, base_price * price_variation)  # Min 30% discount
                
                revenue = quantity * current_price
                
                sales_records.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'product_id': product['product_id'],
                    'product_name': product['product_name'],
                    'category': product['category'],
                    'quantity_sold': quantity,
                    'unit_price': round(current_price, 2),
                    'total_revenue': round(revenue, 2)
                })
                
                price_records.append({
                    'date': date.strftime('%Y-%m-%d'),
                    'product_id': product['product_id'],
                    'product_name': product['product_name'],
                    'current_price': round(current_price, 2),
                    'price_change': round((current_price - base_price) / base_price * 100, 2)
                })
        
        self.sales_data = pd.DataFrame(sales_records)
        self.price_data = pd.DataFrame(price_records)
        
        # Save to CSV
        self.sales_data.to_csv('sample_sales_data.csv', index=False)
        self.price_data.to_csv('sample_price_data.csv', index=False)
        
        print(f"📊 Generated {len(self.sales_data)} sales records")
        print(f"💰 Generated {len(self.price_data)} price records")

# Initialize data manager
data_manager = BusinessDataManager()

def get_product_price_history(product_id: str, days: int = 30) -> str:
    """
    Fetches historical price data for a specific product.
    """
    try:
        price_df = data_manager.price_data
        price_df['date'] = pd.to_datetime(price_df['date'])
        
        # Filter for the specific product and time period
        end_date = price_df['date'].max()
        start_date = end_date - timedelta(days=days)
        
        product_data = price_df[
            (price_df['product_id'] == product_id) & 
            (price_df['date'] >= start_date)
        ].sort_values('date')
        
        if len(product_data) == 0:
            return json.dumps({"error": f"No price data found for product {product_id}"})
        
        # Calculate statistics
        prices = product_data['current_price'].tolist()
        current_price = prices[-1] if prices else 0
        min_price = product_data['current_price'].min()
        max_price = product_data['current_price'].max()
        avg_price = product_data['current_price'].mean()
        
        # Price trend analysis
        recent_prices = product_data.tail(7)['current_price']
        previous_prices = product_data.iloc[-14:-7]['current_price'] if len(product_data) > 14 else recent_prices
        
        recent_avg = recent_prices.mean()
        previous_avg = previous_prices.mean() if len(previous_prices) > 0 else recent_avg
        
        if recent_avg > previous_avg:
            trend = "increasing"
        elif recent_avg < previous_avg:
            trend = "decreasing"
        else:
            trend = "stable"
        
        price_history = product_data[['date', 'current_price']].tail(10).to_dict('records')
        
        response_data = {
            "product_id": product_id,
            "product_name": product_data['product_name'].iloc[0],
            "analysis_period": f"Last {days} days",
            "current_price": round(float(current_price), 2),
            "price_statistics": {
                "minimum_price": round(float(min_price), 2),
                "maximum_price": round(float(max_price), 2),
                "average_price": round(float(avg_price), 2),
                "price_change_percentage": round(((current_price - prices[0]) / prices[0]) * 100, 2) if prices[0] != 0 else 0,
                "price_volatility": round((max_price - min_price) / avg_price * 100, 2) if avg_price != 0 else 0
            },
            "price_trend": trend,
            "data_points": len(product_data),
            "price_history_sample": [
                {
                    "date": row['date'].strftime('%Y-%m-%d'),
                    "price": round(row['current_price'], 2)
                } for row in price_history
            ]
        }
        
        return json.dumps(response_data, indent=2)
        
    except Exception as e:
        return json.dumps({"error": f"Failed to fetch price history: {str(e)}"})

def get_sales_data(product_id: str = None, timeframe: str = "30d") -> str:
    """
    Fetches sales data for specific product or all products.
    """
    try:
        sales_df = data_manager.sales_data
        sales_df['date'] = pd.to_datetime(sales_df['date'])
        
        # Calculate date range
        end_date = sales_df['date'].max()
        days_map = {"7d": 7, "30d": 30, "90d": 90, "1y": 365}
        days = days_map.get(timeframe, 30)
        start_date = end_date - timedelta(days=days)
        
        filtered_df = sales_df[sales_df['date'] >= start_date]
        
        if product_id:
            # Single product analysis
            product_data = filtered_df[filtered_df['product_id'] == product_id]
            
            if len(product_data) == 0:
                return json.dumps({"error": f"No sales data found for {product_id}"})
            
            # Calculate metrics
            total_units = product_data['quantity_sold'].sum()
            total_revenue = product_data['total_revenue'].sum()
            avg_daily_sales = total_units / days
            
            # Sales trend
            recent_sales = product_data.tail(7)['quantity_sold']
            previous_sales = product_data.iloc[-14:-7]['quantity_sold'] if len(product_data) > 14 else recent_sales
            
            recent_avg = recent_sales.mean()
            previous_avg = previous_sales.mean() if len(previous_sales) > 0 else recent_avg
            
            if recent_avg > previous_avg:
                sales_trend = "increasing"
            elif recent_avg < previous_avg:
                sales_trend = "decreasing"
            else:
                sales_trend = "stable"
            
            # Last 7 days data
            last_7_days = []
            for i in range(7, 0, -1):
                date = end_date - timedelta(days=i)
                date_str = date.strftime('%Y-%m-%d')
                day_data = product_data[product_data['date'] == date]
                units = day_data['quantity_sold'].sum() if len(day_data) > 0 else 0
                revenue = day_data['total_revenue'].sum() if len(day_data) > 0 else 0
                
                last_7_days.append({
                    "date": date_str,
                    "units_sold": int(units),
                    "revenue": round(float(revenue), 2)
                })
            
            response_data = {
                "product_id": product_id,
                "product_name": product_data['product_name'].iloc[0],
                "category": product_data['category'].iloc[0],
                "timeframe": timeframe,
                "total_units_sold": int(total_units),
                "total_revenue": round(float(total_revenue), 2),
                "average_daily_sales": round(float(avg_daily_sales), 2),
                "sales_trend": sales_trend,
                "data_points": len(product_data),
                "sales_data_last_7_days": last_7_days
            }
            
        else:
            # All products summary
            product_summary = filtered_df.groupby(['product_id', 'product_name']).agg({
                'quantity_sold': 'sum',
                'total_revenue': 'sum',
                'unit_price': 'mean'
            }).reset_index()
            
            total_all_units = product_summary['quantity_sold'].sum()
            product_summary['market_share'] = round(
                (product_summary['quantity_sold'] / total_all_units) * 100, 2
            )
            
            product_summary = product_summary.rename(columns={'unit_price': 'average_price'})
            product_summary = product_summary.sort_values('total_revenue', ascending=False)
            
            products_list = product_summary.to_dict('records')
            
            response_data = {
                "timeframe": timeframe,
                "total_products": len(product_summary),
                "total_units_sold": int(total_all_units),
                "total_revenue": round(float(product_summary['total_revenue'].sum()), 2),
                "top_performing_products": products_list[:3],
                "all_products_summary": products_list
            }
        
        return json.dumps(response_data, indent=2)
        
    except Exception as e:
        return json.dumps({"error": f"Failed to fetch sales data: {str(e)}"})

def calculate_demand_trend(product_id: str, analysis_period: str = "30d") -> str:
    """
    Analyzes demand trends and patterns for a product.
    """
    try:
        # Get sales data for analysis
        sales_json = get_sales_data(product_id, analysis_period)
        sales_data = json.loads(sales_json)
        
        if "error" in sales_data:
            return sales_json
        
        # Get additional historical data for trend analysis
        sales_df = data_manager.sales_data
        sales_df['date'] = pd.to_datetime(sales_df['date'])
        
        product_data = sales_df[sales_df['product_id'] == product_id]
        
        if len(product_data) == 0:
            return json.dumps({"error": f"No data found for product {product_id}"})
        
        # Calculate weekly trends
        product_data['week'] = product_data['date'].dt.isocalendar().week
        weekly_sales = product_data.groupby('week').agg({
            'quantity_sold': 'sum',
            'total_revenue': 'sum'
        }).reset_index()
        
        # Growth rate calculation
        if len(weekly_sales) >= 2:
            recent_week = weekly_sales.iloc[-1]['quantity_sold']
            previous_week = weekly_sales.iloc[-2]['quantity_sold']
            growth_rate = ((recent_week - previous_week) / previous_week * 100) if previous_week > 0 else 0
        else:
            growth_rate = 0
        
        # Demand classification
        if growth_rate > 25:
            demand_level = "HIGH_GROWTH"
            recommendation = "🚀 High growth - Increase inventory and capitalize on momentum"
        elif growth_rate > 10:
            demand_level = "GROWING" 
            recommendation = "📈 Steady growth - Maintain current strategy"
        elif growth_rate > -5:
            demand_level = "STABLE"
            recommendation = "✅ Stable demand - Maintain inventory levels"
        elif growth_rate > -15:
            demand_level = "DECLINING"
            recommendation = "⚠️ Declining - Consider promotions or marketing"
        else:
            demand_level = "LOW_DEMAND"
            recommendation = "🔻 Low demand - Review product strategy"
        
        # Seasonality analysis
        product_data['month'] = product_data['date'].dt.month
        monthly_variation = product_data.groupby('month')['quantity_sold'].std().mean()
        seasonality = "HIGH" if monthly_variation > 20 else "MODERATE" if monthly_variation > 10 else "LOW"
        
        response_data = {
            "product_id": product_id,
            "product_name": product_data['product_name'].iloc[0],
            "analysis_period": analysis_period,
            "demand_analysis": {
                "demand_level": demand_level,
                "growth_rate_percentage": round(growth_rate, 2),
                "seasonality": seasonality,
                "weekly_variation": round(weekly_sales['quantity_sold'].std(), 2),
                "stability": "HIGH" if abs(growth_rate) < 10 else "MODERATE" if abs(growth_rate) < 20 else "LOW"
            },
            "forecast_insights": {
                "outlook": "POSITIVE" if growth_rate > 5 else "NEUTRAL" if growth_rate > -5 else "CAUTIOUS",
                "recommendation": recommendation,
                "confidence": "HIGH" if len(weekly_sales) > 8 else "MEDIUM"
            },
            "weekly_trend": weekly_sales.tail(4).to_dict('records')
        }
        
        return json.dumps(response_data, indent=2)
        
    except Exception as e:
        return json.dumps({"error": f"Failed to calculate demand trend: {str(e)}"})



# Create the Business Intelligence Agent with tools
business_agent = LlmAgent(
    name="business_intelligence_analyst",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A specialized agent for product pricing, sales analysis, and demand forecasting using real sample data.",
    instruction="""You are BusinessIntelligence AI - an expert in product analysis, pricing strategy, and demand forecasting.

CORE CAPABILITIES:
1. PRICE ANALYSIS - Historical price tracking, trend identification, volatility analysis
2. SALES ANALYTICS - Sales performance, revenue tracking, product comparison  
3. DEMAND FORECASTING - Trend analysis, growth rates, seasonality detection
4. BUSINESS INSIGHTS - Data-driven recommendations and strategic insights

AVAILABLE TOOLS:
- get_product_price_history: Use for price history, trends, and pricing strategy
- get_sales_data: Use for sales performance, revenue analysis, and product comparison
- calculate_demand_trend: Use for demand forecasting, growth analysis, and inventory planning

RESPONSE GUIDELINES:
🎯 START with clear executive summary
📊 PRESENT key metrics and trends
📈 HIGHLIGHT important patterns and anomalies
💡 PROVIDE actionable recommendations
🔢 USE specific numbers and percentages
📋 STRUCTURE responses for business audience

TOOL SELECTION GUIDE:
• When user asks about prices, costs, or pricing strategy → Use get_product_price_history
• When user asks about sales, revenue, or performance → Use get_sales_data  
• When user asks about demand, trends, or forecasting → Use calculate_demand_trend
• For comprehensive analysis → Use multiple tools

EXAMPLE QUERIES:
"Analyze price history for PROD-001 over 60 days" → get_product_price_history
"Compare sales performance of all products last quarter" → get_sales_data
"What's the demand trend for PROD-002 and should we increase inventory?" → calculate_demand_trend + get_sales_data
"Show me comprehensive analysis for PROD-003" → All three tools

Always be data-driven, analytical, and business-focused in your responses.""",
    tools=[]  
)

print("✅ Business Intelligence Agent created successfully!")
print("✅ Tools integrated: Price History, Sales Data, Demand Trend")
print("✅ Sample data loaded and ready for analysis!")

📊 Generated 905 sales records
💰 Generated 905 price records
✅ Generated new sample data
✅ Business Intelligence Agent created successfully!
✅ Tools integrated: Price History, Sales Data, Demand Trend
✅ Sample data loaded and ready for analysis!


In [7]:
runner = InMemoryRunner(agent = business_agent)
print("runner Created")

runner Created


In [8]:
response = await runner.run_debug(
    "Can you give me the price of any product you have of product ID PROD-001"
)


 ### Created new session: debug_session_id

User > Can you give me the price of any product you have of product ID PROD-001


In [9]:
response = await runner.run_debug(
"yesterday")


 ### Continue session: debug_session_id

User > yesterday
